In [1]:
import pandas as pd
import numpy as np
import random
import pickle

In [ ]:
# 1 way queries

# df = pd.read_csv("ppmf_01.csv") # to read the actual census df

m = 10  # num cols
n = 100 # num rows
df = pd.DataFrame(np.random.randint(0,2,size=(n, m)))

def reconstruct(df):
    # tabulate 1 way queries
    qs = df.sum()
    recon = pd.DataFrame(columns=df.columns)
    (n, m) = df.shape
    
    for _ in range(len(df)):
        row = construct_row(recon, qs, n, m)
        recon.loc[len(recon)] = row
        qs = qs - row
        
    return recon
        
    
def construct_row(recon, qs, n, m):
    prob = qs/(n - len(recon))
    row = np.array([])

    for i in range(m):
        if random.random() <= prob[i]:
            row = np.append(row, 1)
        else:
            row = np.append(row, 0)

    row = row.astype(int)
    return row

recon = reconstruct(df)
recon.sum()
df.sum()

### Two Way Queries

In [ ]:
class OneWayQuery:
    def __init__(self, col, q, n):
        self.col = col
        self.q = q
        self.n = n

class TwoWayQuery:
    def __init__(self, col_1, col_2, q, n):
        self.col_1 = col_1
        self.col_2 = col_2
        self.q = q
        self.n = n

In [ ]:
m = 2  # num cols
n = 7 # num rows
df = pd.DataFrame(np.random.randint(0,2,size=(n, m)))
df

recon = pd.DataFrame(np.empty((n,m)))
recon

col_1 = 0
col_2 = 1
ans = len(df.query(f'@df[{col_1}] == 1 and @df[{col_2}] == 1'))
two = TwoWayQuery(col_1, col_2, ans, n)

ones = []
for index, ans in df.sum().items():
    q = OneWayQuery(index, ans, n)
    ones.append(q)


for i in range(n):
    for j in range(m):
        if j == 0:
            if random.random() <= (ones[j].q / ones[j].n):
                recon.loc[i, j] = 1
                ones[j].q -= 1
                ones[j].n -= 1
            else:
                recon.loc[i, j] = 0
                ones[j].n -= 1
        else: # j = 1
            if random.random() <= (ones[1].q - two.q)/(max(ones[0].n - ones[0].q, 1)):
                recon.loc[i, j] = 1
                ones[j].q -= 1
                ones[j].n -= 1
                
                if df.loc[i, 0] == 1: # when 2 way query row is made
                    two.q -= 1
                    two.n -= 1
            else:
                recon.loc[i, j] = 0
                ones[j].n -= 1

recon[0] = recon[0].astype(int)
recon[1] = recon[1].astype(int)

In [21]:
df = pd.read_csv("./dp-query-release/datasets/ppmf/ppmf_01097007000.csv")
df = df[['TABBLK', 'CENRACE', 'CENHISP', 'QAGE', 'QSEX']]
df

,TABBLK,CENRACE,CENHISP,QAGE,QSEX
0,0,0,1,58,1
1,0,0,1,58,1
2,0,0,1,58,1
3,0,0,1,58,1
4,0,0,1,58,1
...,...,...,...,...,...
4532,48,0,1,12,0
4533,48,0,1,12,0
4534,48,2,1,28,0
4535,48,0,1,51,0


In [22]:
def turn_ones(x):
    if x is True:
        return 1
    else:
        return 0

df["QAGE"] = df["QAGE"].apply(lambda x: x >= 18).apply(turn_ones) 
df["CENRACE"] = df["CENRACE"].apply(lambda x: x if x <= 5 else 6)
df

,TABBLK,CENRACE,CENHISP,QAGE,QSEX
0,0,0,1,1,1
1,0,0,1,1,1
2,0,0,1,1,1
3,0,0,1,1,1
4,0,0,1,1,1
...,...,...,...,...,...
4532,48,0,1,0,0
4533,48,0,1,0,0
4534,48,2,1,1,0
4535,48,0,1,1,0


In [4]:
num_race = 2
num_hisp = 2
num_sex = 2
num_age = 2

# the counts are in [num_race, num_hisp, num_age, num_sex]

In [5]:
col_block = "TABBLK"
col_age = "QAGE"
col_sex = "QSEX"
col_race = "CENRACE"
col_hisp = "CENHISP"

num_blocks = len(df[col_block].unique())
queries = np.ndarray([num_blocks, num_race, num_hisp, num_age, num_sex])

for block in sorted(df[col_block].unique()):
    print(block)
    for race in range(num_race):
        for hisp in range(num_hisp):
            for sex in range(num_sex):
                for age in range(num_age):          
                    queries[block, race, hisp, age, sex] = len(df.query(f'{col_block} == {block} and ' +
                                                                        f'{col_race} == {race} and ' +
                                                                        f'{col_hisp} == {hisp} and ' +
                                                                        f'{col_sex} == {sex} and ' +
                                                                        f'{col_age} == {age}'))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113


In [8]:
def num_ppl_of_age(block, age, num_race=2, num_hisp=2, num_sex=2):
    num_ppl = 0

    for r in range(num_race):
        for h in range(num_hisp):
            for s in range(num_sex):
                num_ppl += block[r, h, age, s]
    
    return num_ppl 

def num_ppl_of_age_and_sex(block, age, sex, num_race=2, num_hisp=2):
    num_ppl = 0

    for r in range(num_race):
        for h in range(num_hisp):
            num_ppl += block[r, h, age, sex]
    
    return num_ppl 

def construct_row(block, verbose=False):
    """
    """
    race_prob = (block[0].sum() / block.sum())        
    recon_race = 0 if random.random() <= race_prob else 1

    hisp_prob = (block[recon_race, 0].sum() / block[recon_race].sum())        
    recon_hisp = 0 if random.random() <= hisp_prob else 1
        
    age_prob = (block[recon_race, recon_hisp, 0].sum() / block[recon_race, recon_hisp].sum())
    recon_age = 0 if random.random() <= age_prob else 1

    sex_prob = (num_ppl_of_age_and_sex(block, recon_age, 0) / num_ppl_of_age(block, recon_age))         
    recon_sex = 0 if random.random() <= sex_prob else 1
    
    if verbose:
        print()
        print("Pr(Race = 0): ", race_prob)
        print(f'Pr(Hisp = 0| Race = {recon_race}): ', hisp_prob)
        print(f'Pr(Age = 0| Hisp = {recon_hisp} & Race = {recon_race}): ', age_prob)
        print(f'Pr(Sex = 0| Age = {recon_age}): ', sex_prob)
        print("(Race, Hisp, Age, Sex) = (", recon_race, ",", recon_hisp, ",", recon_age, ",", recon_sex, ")")
        print()
    
    return (recon_race, recon_hisp, recon_age, recon_sex)

In [9]:
blk_id = 2
recon_arr = np.zeros([num_blocks, num_race, num_hisp, num_age, num_sex])
recon_df = pd.DataFrame(columns=["TABBLK", "CENRACE", "CENHISP", "QAGE", "QSEX"])
block = queries[blk_id].copy()

num_ppl = int(block.sum())
for i in range(num_ppl):
    (recon_race, recon_hisp, recon_age, recon_sex) = construct_row(block, True)
    recon_df.loc[len(recon_df)] = [blk_id, recon_race, recon_hisp, recon_age, recon_sex]
    
    recon_arr[blk_id, recon_race, recon_hisp, recon_age, recon_sex] += 1
    block[recon_race, recon_hisp, recon_age, recon_sex] -= 1


Pr(Race = 0):  0.6
Pr(Hisp = 0| Race = 1):  0.0
Pr(Age = 0| Hisp = 1 & Race = 1):  0.0
Pr(Sex = 0| Age = 1):  0.6
(Race, Hisp, Age, Sex) = ( 1 , 1 , 1 , 0 )


Pr(Race = 0):  0.75
Pr(Hisp = 0| Race = 0):  0.0
Pr(Age = 0| Hisp = 1 & Race = 0):  0.0
Pr(Sex = 0| Age = 1):  0.5
(Race, Hisp, Age, Sex) = ( 0 , 1 , 1 , 0 )


Pr(Race = 0):  0.6666666666666666
Pr(Hisp = 0| Race = 0):  0.0
Pr(Age = 0| Hisp = 1 & Race = 0):  0.0
Pr(Sex = 0| Age = 1):  0.3333333333333333
(Race, Hisp, Age, Sex) = ( 0 , 1 , 1 , 0 )


Pr(Race = 0):  0.5
Pr(Hisp = 0| Race = 1):  0.0
Pr(Age = 0| Hisp = 1 & Race = 1):  0.0
Pr(Sex = 0| Age = 1):  0.0
(Race, Hisp, Age, Sex) = ( 1 , 1 , 1 , 1 )


Pr(Race = 0):  1.0
Pr(Hisp = 0| Race = 0):  0.0
Pr(Age = 0| Hisp = 1 & Race = 0):  0.0
Pr(Sex = 0| Age = 1):  0.0
(Race, Hisp, Age, Sex) = ( 0 , 1 , 1 , 1 )



In [11]:
races = []

In [23]:
df["CENRACE"].unique()

array([0, 6, 1, 2, 5, 3, 4])

In [109]:
def query_manager(block):
    """
    """
    num_race = 7
    num_hisp = 2
    num_sex = 2
    num_age = 2
    
    races = np.zeros(num_race)
    for (k, v) in block["CENRACE"].value_counts().items():
        races[k] = v
    
    hisp_race = np.zeros([num_hisp, num_race])
    for i in range(num_hisp):
        for (k, v) in block[block["CENHISP"] == i]["CENRACE"].value_counts().items():
            hisp_race[i, k] = v
        
    nh_race_age = np.zeros([num_race, num_age])
    for race in range(num_race):
        for age in range(num_age):
            nh_race_age[race, age] = len(block.query(f'CENHISP == 0 and CENRACE == {race} and QAGE == {age}'))

    age_sex = np.zeros([num_age, num_sex])
    for age in range(num_age):
        for sex in range(num_sex):
            age_sex[sex, age] = len(block.query(f'QSEX == {sex} and QAGE == {age}'))
        
    return races, hisp_race, nh_race_age, age_sex
        

In [76]:
df["CENRACE"].value_counts()

0    4168
1     249
3      53
2      32
6      27
5       6
4       2
Name: CENRACE, dtype: int64

In [110]:
query_manager(block)

(array([102.,   8.,   3.,   0.,   0.,   0.,   0.]),
 array([[ 4.,  0.,  0.,  0.,  0.,  0.,  0.],
        [98.,  8.,  3.,  0.,  0.,  0.,  0.]]),
 array([[0., 4.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]]),
 array([[10., 48.],
        [ 3., 52.]]))

In [88]:
block = df[df["TABBLK"] == 13]
block

,TABBLK,CENRACE,CENHISP,QAGE,QSEX
418,13,0,1,0,1
419,13,0,1,0,1
420,13,1,1,0,1
421,13,0,1,1,1
422,13,0,1,1,1
...,...,...,...,...,...
526,13,0,1,1,0
527,13,0,1,1,0
528,13,0,1,1,0
529,13,0,1,1,0


In [58]:
block.groupby(["CENRACE"]).count()

TABBLK  QAGE  QSEX
CENHISP CENRACE                    
1       0             3     3     3
        6             2     2     2

In [66]:
for k,v in block[block["CENHISP"] == 1].groupby(["CENRACE"]).count().items():
    print(v)

CENRACE
0    3
6    2
Name: TABBLK, dtype: int64
CENRACE
0    3
6    2
Name: CENHISP, dtype: int64
CENRACE
0    3
6    2
Name: QAGE, dtype: int64
CENRACE
0    3
6    2
Name: QSEX, dtype: int64


In [92]:
for k, v in block[block["CENHISP"] == 1].groupby(["CENRACE", "QAGE"]).count().items():
    print(v)

CENRACE  QAGE
0        0       12
         1       86
1        0        1
         1        7
2        1        3
Name: TABBLK, dtype: int64
CENRACE  QAGE
0        0       12
         1       86
1        0        1
         1        7
2        1        3
Name: CENHISP, dtype: int64
CENRACE  QAGE
0        0       12
         1       86
1        0        1
         1        7
2        1        3
Name: QSEX, dtype: int64


In [74]:
np.zeros([2, 2])

array([[0., 0.],
       [0., 0.]])

In [108]:
race = 0
age = 0
block.query(f'CENHISP == 1 and CENRACE == {race} and QAGE == {age}')

,TABBLK,CENRACE,CENHISP,QAGE,QSEX
418,13,0,1,0,1
419,13,0,1,0,1
473,13,0,1,0,0
474,13,0,1,0,0
475,13,0,1,0,0
476,13,0,1,0,0
477,13,0,1,0,0
478,13,0,1,0,0
479,13,0,1,0,0
480,13,0,1,0,0


In [107]:
block

,TABBLK,CENRACE,CENHISP,QAGE,QSEX
418,13,0,1,0,1
419,13,0,1,0,1
420,13,1,1,0,1
421,13,0,1,1,1
422,13,0,1,1,1
...,...,...,...,...,...
526,13,0,1,1,0
527,13,0,1,1,0
528,13,0,1,1,0
529,13,0,1,1,0
